In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.10.2") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()
import synapse.ml
from synapse.ml.opencv import *
import os
import time

In [2]:
!pip install tensorflow
!pip install numpy --upgrade
!pip install torch
!pip install torchvision
!pip install opencv-python

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import functools
import torch
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.datasets.folder import default_loader 


/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [4]:
class ImageDataset(Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, index):
        image = default_loader(self.paths[index])
        if self.transform is not None:
            image = self.transform(image)
        return image


In [5]:
device = torch.device("cpu")

In [6]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
  ])


In [7]:
def get_model_for_eval():
    model_state = models.resnet50(pretrained=True).state_dict()
    model = models.resnet50(pretrained=True)
    model.load_state_dict(model_state)
    model.eval()
    return model


In [8]:
import pandas as pd
def predict_batch_udf(paths: pd.Series) -> pd.Series:
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
      ])
    images = ImageDataset(paths, transform=transform)
    loader = torch.utils.data.DataLoader(images, batch_size=2, num_workers=1)
    model = get_model_for_eval()
    model.to(device)
    all_predictions = []
    with torch.no_grad():
        for batch in loader:
            predictions = list(model(batch.to(device)).cpu().numpy())
            for prediction in predictions:
                prediction_lis = [float(temp) for temp in prediction]
                all_predictions.append(list(prediction_lis))
    return all_predictions


In [9]:
dataset_dir = "./export/"
train_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(dataset_dir) if ".ipynb_checkpoints" not in dp for f in filenames if os.path.splitext(f)[1] == '.jpg']
len(train_files)


29800

In [10]:
tik = time.time()
model = get_model_for_eval()
model.to(device)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
  ])
images = ImageDataset(train_files, transform=transform)
loader = torch.utils.data.DataLoader(images, batch_size=20, num_workers=2)
all_predictions = []
with torch.no_grad():
    for batch in loader:
        predictions = list(model(batch.to(device)).cpu().numpy())
        for prediction in predictions:
            prediction_lis = [float(temp) for temp in prediction]
            all_predictions.append(list(prediction_lis))
vector_train = all_predictions
tok = time.time() -tik

/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
print(tok)

3461.8485560417175


In [12]:
dataset_dir = "./images/test/"

test_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(dataset_dir) if ".ipynb_checkpoints" not in dp for f in filenames if os.path.splitext(f)[1] == '.png']
print(test_files)

['./images/test/road0.png']


In [13]:
model = get_model_for_eval()
model.to(device)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
  ])
images = ImageDataset(test_files, transform=transform)
loader = torch.utils.data.DataLoader(images, batch_size=20, num_workers=2)
all_predictions = []
with torch.no_grad():
    for batch in loader:
        predictions = list(model(batch.to(device)).cpu().numpy())
        for prediction in predictions:
            prediction_lis = [float(temp) for temp in prediction]
            all_predictions.append(list(prediction_lis))
vector_test = all_predictions

In [14]:
train_df_sample = train_df.limit(1)

NameError: name 'train_df' is not defined

In [35]:
train_df_sample.collect()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [28]:
from pyspark.sql.functions import col, concat_ws
from pyspark.ml.functions import array_to_vector
columns = ['id', 'vector']  
train_df = spark.createDataFrame(zip(train_files, vector_train), columns)
train_df = train_df.withColumn("feature",array_to_vector("vector"))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space


In [15]:
from pyspark.ml.functions import array_to_vector
columns = ['id', 'vector']  
test_df = spark.createDataFrame(zip(test_files, vector_test), columns)
test_df = test_df.withColumn("feature",array_to_vector("vector"))
test_df.dtypes

[('id', 'string'), ('vector', 'array<double>'), ('feature', 'vector')]

In [16]:
from pyspark.sql import SparkSession

# Bootstrap Spark Session
spark = SparkSession.builder.getOrCreate()

from synapse.ml.core.platform import running_on_synapse

if running_on_synapse():
    from notebookutils.visualization import display


In [33]:
train_df_sample.count()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [17]:
tik = time.time()
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import BucketedRandomProjectionLSH


key = Vectors.dense(test_df.collect()[0][1])
brp = BucketedRandomProjectionLSH(inputCol="feature", outputCol="hashes", bucketLength=2.0,
                                  numHashTables=1)
model = brp.fit(train_df)

# Feature Transformation
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(train_df)

# Compute the locality sensitive hashes for the input rows, then perform approximate
# similarity join.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxSimilarityJoin(transformedA, transformedB, 1.5)`
print("Approximately joining dfA and dfB on Euclidean distance smaller than 1.5:")
output_df_1 = model.approxSimilarityJoin(train_df, test_df, 50, distCol="EuclideanDistance")\
    .select(col("datasetA.id").alias("idA"),
            col("datasetB.id").alias("idB"),
            col("EuclideanDistance"))

# Compute the locality sensitive hashes for the input rows, then perform approximate nearest
# neighbor search.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxNearestNeighbors(transformedA, key, 2)`
print("Approximately searching dfA for 2 nearest neighbors of the key:")
output_df_2 = model.approxNearestNeighbors(train_df, key, 100)
print(time.time()-tik)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt

KeyboardInterrupt



In [ ]:
output_df_1 = output_df_1.orderBy(col("EuclideanDistance").asc())
output_df_1.limit(10).show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import io
from PIL import Image

print("Input Images")
for i in test_files:
    img = Image.open(i)
    plt.imshow(img)
    plt.show()


In [ ]:
images_list = output_df_1.select(col("idA")).limit(5).collect()
print("Output Images using EuclideanDistance")
for i in images_list:
    images = spark.read.format("image").option("dropInvalid", True).load(i[0])
    im = images.collect()[0][0][0]
    img = Image.open("/"+im.split("///")[1])
    plt.imshow(img)
    plt.show()

In [ ]:
images_list_2 = output_df_2.select(col("id")).limit(10).collect()
print("Output Images using Nearest Neighbours")
for i in images_list_2:
    images = spark.read.format("image").option("dropInvalid", True).load(i[0])
    im = images.collect()[0][0][0]
    img = Image.open("/"+im.split("///")[1])
    plt.imshow(img)
    plt.show()

In [ ]:
'''
import synapse.ml
import numpy as np
from synapse.ml.opencv import toNDArray
from synapse.ml.io import *

imageDir = "images"
images = spark.read.image().load(imageDir).cache()

#df = spark.read.format("image").load("images/traffic_signal.jpeg")

images.printSchema()
print(images.count())

from synapse.ml.core.platform import running_on_binder

if running_on_binder():
    from IPython import get_ipython
from PIL import Image
import matplotlib.pyplot as plt

data = images.take(2)  # take first three rows of the dataframe
im = data[1][0]  # the image is in the first column of a given row

print("image type: {}, number of fields: {}".format(type(im), len(im)))
print("image path: {}".format(im.origin))
print("height: {}, width: {}, OpenCV type: {}".format(im.height, im.width, im.mode))

arr = toNDArray(im)  # convert to numpy array
print(images.count())
plt.imshow(Image.fromarray(arr, "RGB"))

from synapse.ml.core.platform import running_on_binder

if running_on_binder():
    from IPython import get_ipython
from PIL import Image
import matplotlib.pyplot as plt

data = images.take(2)  # take first three rows of the dataframe
im = data[1][0]  # the image is in the first column of a given row

print("image type: {}, number of fields: {}".format(type(im), len(im)))
print("image path: {}".format(im.origin))
print("height: {}, width: {}, OpenCV type: {}".format(im.height, im.width, im.mode))

arr = toNDArray(im)  # convert to numpy array
print(images.count())
plt.imshow(Image.fromarray(arr, "RGB"))


import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT, ArrayType

ImageSchema.imageFields

img2vec = F.udf(lambda x: DenseVector(ImageSchema.toNDArray(x).flatten()[0:10]), VectorUDT())


df = df.withColumn('vecs', img2vec("image"))
df.show()


from pyspark.ml.functions import vector_to_array
df = df.withColumn('features', vector_to_array('vecs'))
'''